# ShortCut Solver

Very similar to example 01-Basic. But this time with sparse data structure for coupling handler and cost function. Almost no memory demand.

In [ ]:
# load libraries
from lib.header_notebook import *

# assume that CPLEX back-end is installed
import Solvers.ShortCutSolver as ShortCutSolver
import Solvers.ShortCutSolver_CPLEX as ShortCutSolver_CPLEX

%matplotlib inline

## Problem Setup

In [ ]:
# load two 128x128 images
imgX=sciio.loadmat("data/density-img/f-000-128.mat")["a"]
imgY=sciio.loadmat("data/density-img/f-001-128.mat")["a"]

# preprocessing (add small constant background mass, normalize masses, extract geometric positions of pixels)
(muX,posX)=OTTools.processDensity_Grid(imgX,totalMass=1.,constOffset=1E-6)
(muY,posY)=OTTools.processDensity_Grid(imgY,totalMass=1.,constOffset=1E-6)

In [ ]:
# visualize images
fig=plt.figure(figsize=(8,4))
fig.add_subplot(1,2,1)
plt.imshow(imgX)
fig.add_subplot(1,2,2)
plt.imshow(imgY)
plt.show()

In [ ]:
# set up hierarchical partitions

# finest layer above image has 2^partitionDepth grid points per dimension, then one below is image
partitionDepth=6
# another partition parameter, to be discussed later
partitionChildMode=HierarchicalPartition.THPMode_Grid

# create partitions from point clouds & measures, export partitions already to c++ library for later use
(partitionX,pointerX)=HierarchicalPartition.GetPartition(posX,partitionDepth,partitionChildMode,imgX.shape, mu=muX,\
    signal_pos=True, signal_radii=False,clib=SolverCFC, export=True, verbose=False)

(partitionY,pointerY)=HierarchicalPartition.GetPartition(posY,partitionDepth,partitionChildMode,imgY.shape, mu=muY,\
    signal_pos=True, signal_radii=True,clib=SolverCFC, export=True, verbose=False)

pointerYpos=HierarchicalPartition.getSignalPointer(partitionY,"pos")
pointerYradii=HierarchicalPartition.getSignalPointer(partitionY,"radii", lBottom=partitionY.nlayers-2)

# prepare some aux information for sparse shielding method
cList=[(partitionX.layers[nLayer]["pos"],partitionY.layers[nLayer]["pos"])
        for nLayer in range(partitionX.nlayers)]


# print a few stats on the created problem
print("cells in partition x: ", partitionX.cardLayers)
print("cells in partition y: ", partitionY.cardLayers)

## Solving

### Multiscale Solving

In [ ]:
# the algorithm has a modular structure. different components can be combined to final algorithm

# refinement:
#     generate initial fine coupling support when doing a layer refinement
methodSetup_Refinement=ShortCutSolver.getMethodSetup_Refinement(pointerX,pointerY,SolverCFC)

# coupling handler:
#     create a fully sparse coupling handler that only allocates required entries and computes
#     sparse cost on demand (only implemented for |x-y|^2 so far, but easy to extend)

methodSetup_CouplingHandler=ShortCutSolver.getMethodSetup_CouplingHandler_Sparse_dynamicC(\
        ShortCutSolver.Setup_CostFunctionProvider_SqrEuclidean)

# solver for sparse sub-problems
#     in this example only use CPLEX solver (Lemon requires some more preprocessing of densities)
#     couplingHandlerType must match the coupling handler chosen above (this time sparse variant, compare to example 01)
#     initializeBases=True indicates that warm-starting the solver during iterations on same scale will be used.
methodSetup_SubSolver=ShortCutSolver_CPLEX.getMethodSetup_SubSolver_CPLEX(\
        couplingHandlerType=ShortCutSolver_CPLEX.CH_Sparse,initializeBases=True)


# shielding
#     one must choose a shielding method that matches the geometry of the cost
#     in this example we have the simplest possible case: squared Euclidean distance on regular Cartesian grid.
#     so can choose simplest (and fastest) shielding method

methodSetup_Shielding=ShortCutSolver.getMethodSetup_Shielding_Grid()

In [ ]:
# do multi-scale solving.
#     algorithm gets all the chosen methods above and combines them into full ShortCut solver.
#     solves successively from very coarse level to finest level
#     is configured in verbose mode. at each level a small report is printed
time1=datetime.datetime.now()
result=ShortCutSolver.MultiscaleSolver(partitionX, partitionY, cList,\
    methodSetup_Refinement, methodSetup_CouplingHandler, methodSetup_SubSolver, methodSetup_Shielding,\
    nLayerInitial=1,nLayerFinal=None,\
    Verbose=True,\
    maxSteps=100,collectReports=True,measureTimes=True,stepwiseAnalysis=False)
time2=datetime.datetime.now()
print(time2-time1)

#### Verify Shielding Condition and Optimality

In [ ]:
# no longer really elegantly feasible on sparse data structure (cost is only computed on demand)

#### Check Objective

In [ ]:
# extract sparse data of optimal coupling, returns basically (data,indices,indptr)
# for scipy.sparse.csr_sparse array of mu
# full neighbourhood N upon termination of algorithm is returned
mu=ShortCutSolver.CouplingHandler_Sparse_GetMu(result[0]["pointer_couplinghandler"])
# csimilarly: cost entries on current neighbourhood N
costdata=ShortCutSolver.CouplingHandler_Sparse_GetCost(result[0]["pointer_couplinghandler"],mu[0].shape[0])

In [ ]:
# dual potentials remain the same as in dense data structure
alpha=result[0]["result_subsolver"]["alpha"]
beta=result[0]["result_subsolver"]["beta"]

In [ ]:
# primal score
np.sum(mu[0]*costdata)

In [ ]:
# dual score
np.sum(alpha*muX)+np.sum(beta*muY)

In [ ]:
# extract sparse representation of optimal coupling, take only non-zero values
muSupport=ShortCutSolver.CouplingHandler_Sparse_GetSupport(result[0]["pointer_couplinghandler"])

In [ ]:
# evaluate cost function on entries of muSupport
costdataSupport=ShortCutSolver.CostFunctionProvider_Evaluate(result[0]["result_couplinghandler"]["pointerCFP"],\
        muSupport[1],muSupport[2])

In [ ]:
np.sum(muSupport[0]*costdataSupport)

### Dense Solver

In [ ]:
# no longer really feasible, will in general need to much memory and require about 1h at 128x128 images

### Cleaning Up

In [ ]:
ShortCutSolver.SolverClose(result[0]["pointer"])
SolverCFC.Close(pointerX)
SolverCFC.Close(pointerY)